In [ ]:
import argparse   #argparse 
import os 
#os模块是用于访问操作系统功能的模块；使用OS模块提供的接口实现跨平台访问。
#os目录操作-增删改查 os.listdir();os.mkdir();os.rmdir();
#os.path模块，用来返回文件路径或大小，或者连接目录与文件名或目录
import matplotlib.pyplot as plt 
from matplotlib.pyplot import imshow 
# Matplotlib是类似Matlab的画图工具；Pyplot模块来画图
import scipy.io 
import scipy.misc
#利用scipy的输入输出，来对图像进行操作

import numpy as np 
import pandas as pd 

import PIL 
#PIL：Python Imaging Library 图像处理模块
#包括Open、Save、format、Mode、Convert、Eval、Merge等等

import tensorflow as tf 
from keras import backend as K 
#Keras是模型级库，提供高层次的构建模块，但张量乘积和卷积依赖tf后端

from keras.layers import Input, Lamda, Conv2D
from keras.models import load_model,Model 

from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
#yolo_utils是自定义的函数，从yolo_utils导入 

from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_coners,preprocess_true_boxes,yolo_loss,yolo_body
#yad2k是训练好的模型参数，从文件yad2k.models.keras_yolo导入

%matplotlib inline 

In [ ]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold=.6):
        #Step1 
        box_scores=box_confidence*box_class_probs  
        #Step2
        box_classes=K.argmax(box_scores, axis=-1)  #返回最大数值的坐标
        box_class_scores=K.max(box_scores, axis=-1)
        #Step3 
        filtering_mask = box_class_scores >= threshold
        #Step4
        scores=tf.boolean_mask(box_class_scores,filtering_mask) 
        boxes=tf.boolean_mask(boxes, filtering_mask) 
        classes=tf.boolean_mask(box_classes,filtering_mask) 
        
        return scores,boxes,classes 

In [ ]:
with tf.Session() as test_a: 
    box_confidence = tf.random_normal([19,19,5,1], mean=1, stddev=4, seed=1)
    boxes = tf.random_normal([19,19,5,4], mean=1, stddev=4, seed=1) 
    box_class_probs = tf.random_normal([19,19,5,80], mean=1, stddev=4, seed=1) 
    scores,boxes,classes = yolo_filter_boxes(box_confidence,boxes,box_class_probs, threshold=0.5)
    print("score[2]=" + str(scores[2].eval()))
    print("boxes[2]=" + str（boxes[2].eval())
    print("classes[2]=" + str(classes[2].eval()))
    print("scores.shape=" + str(scores.shape)) 
    print("boxes.shape=" + str(boxes.shape)) 
    print("classes.shape=" + str(classes.shape)) 
    

In [ ]:
def iou(box1, box2):
    #Calculate the coordinates of the intersection of box1 and box2 
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])
    inter_area = (yi2-yi1)*(xi2-xi1) 
    
    #Calculate the Union area by using Formula
    box1_area = (box1[2]-box1[0]) * (box1[3]-box1[1])
    box2_area = (box2[2]-box2[0]) * (box2[3]-box2[1])
    union_area = box1_area + box2_area - inter_area
    iou = inter_area / union_area
    return iou 

In [ ]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold=0.5): 
    max_boxes_tensor = K.variable(max_boxes, dtype='int32') 
    K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
    #Use tf.image.non_max_suppression() to get the list of indices 
    nms_indices = tf.image.non_max_suppression(boxes,scores,max_boxes,iou_threshold)
    
    scores=K.gather(scores, nms_indices)
    boxes=K.gather(boxes,nms_indices)
    classes=K.gather(classes, nms_indices)
    
    return scores, boxes, classes 

In [ ]:
with tf.Session() as test_b: 
    scores = tf.random_normal([54,],mean=1,stddev=4,seed=1)
    boxes = tf.random_normal([54,4],mean=1,stddev=4,seed=1)
    classes = tf.random_normal([54,],mean=1,stddev=4,seed=1) 
    scores, boxes, classes = yolo_non_max_suppression(scores,boxes,classes) 
    print("scores[2] = "+ str(scores[2].eval()))
    print("boxes[2]=" + str(boxes[2].eval()))
    print("classes[2]=" + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape)) 
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))

Test YOLO pretrained model on images 

In [ ]:
#Start by creating a session to start your graph 
sess = K.get_session() 

In [ ]:
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt") 
image_shape = (720., 1280.)

yolo_model = load_model("model_data/yolo.h5")
yolo_model.summary()

#Convert output of the model to usable bounding box tensors 
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

#Filtering boxes 
scores,boxes,classes = yolo_eval(yolo_outputs, image_shape)

In [ ]:
def predict(sess,image_file):
    image, image_data = preprocess_image("images/" + image_file, model_image_size=(608,608))
    out_scores, out_boxes, out_classes = sess.run([scores,boxes,classes],feed_dict={yolo_model.input:image_data,K.learning_phase():0})
    
    #Print predictions info 
    print('Found {} boxes for {}'.format(len(out_boxes), image_file))
    colors=generate_colors(class_names) 
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names,colors) 
    
    #Save the predicted bounding box on the image 
    image.save(os.path.join("out",image_file),quality=90)
    
    #Display the results in the notebook
    output_image = scipy.misc.imread(os.path.join("out",image_file))
    imshow(output_image)
    
    return out_scores, out_boxes, out_classes
    

In [ ]:
out_scores, out_boxes, out_classes = predict(sess,"test.jpg")